In [2]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/

Mounted at /content/drive


#Import thư viện

In [4]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load data


In [5]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/content/drive/MyDrive/ChatBox/data/medical.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()

,Title,Detailed Content,Reference Link
0,Rào cản tự nhiên chống lại nhiễm trùng về Da,Da thường chặn các vi sinh vật xâm nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,Rào cản tự nhiên chống lại nhiễm trùng về Niêm...,Nhiều màng nhầy ngập trong chất tiết có đặc tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Đường hô hấp có các hệ thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Các rào cản trong đường tiêu hóa bao gồm pH ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Các rào cản về đường sinh dục tiết niệu bao gồ...,https://www.msdmanuals.com/vi-vn/professional/...


In [6]:
df.loc[1, 'Detailed Content']

'Nhiều màng nhầy ngập trong chất tiết có đặc tính kháng khuẩn. Ví dụ, chất nhầy cổ tử cung, dịch tuyến tiền liệt và nước mắt có chứa lysozyme, chất này chia cắt liên kết axit muramic trong thành tế bào vi khuẩn, đặc biệt là ở các sinh vật gram dương; vi khuẩn gram âm được bảo vệ bởi lipopolysaccharid ở màng ngoài của chúng.\nCác dịch tiết tại chỗ cũng chứa globulin miễn dịch, chủ yếu là IgG và IgA tiết ra, ngăn ngừa các vi sinh vật gắn vào tế bào chủ, và các protein liên kết sắt, rất cần thiết cho nhiều vi sinh vật.'

In [19]:
# Hàm để chia văn bản thành các đoạn dựa trên dấu câu
def split_into_sentences(text):
    # Kiểm tra xem text có phải là chuỗi không và xử lý NaN
    if isinstance(text, str):
        # Tách văn bản theo dấu chấm câu như ".", "!", "?" và giữ lại dấu câu
        sentences = re.split(r'(?<=[.!?]) +', text)
        return sentences
    else:
        # Trả về danh sách rỗng nếu text không phải là chuỗi hoặc là NaN
        return []

# Hàm tính toán TF-IDF, chọn đoạn có liên quan cao nhất, và tìm vị trí của câu trong ngữ cảnh
def extract_highest_tfidf_sentence_with_position(title, context):
    # Chia context thành các đoạn văn
    sentences = split_into_sentences(context)
    unique_sentences = list(set(sentences))
    # Tạo danh sách kết hợp Title và các đoạn trong context để tính TF-IDF
    documents = [title] + unique_sentences  # Tiêu đề được đặt trước các đoạn văn trong context

    # Khởi tạo TfidfVectorizer để tính toán TF-IDF
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Tính điểm TF-IDF của từng đoạn văn dựa trên sự liên quan với tiêu đề
    title_vector = tfidf_matrix[0].toarray()  # Vector của title
    context_vectors = tfidf_matrix[1:].toarray()  # Vectors của các đoạn trong context

    # Tính độ tương đồng giữa title và từng đoạn context
    similarity_scores = np.dot(context_vectors, title_vector.T).flatten()

    # Tìm chỉ số của câu có điểm TF-IDF cao nhất
    highest_score_index = np.argmax(similarity_scores)

    # Lấy câu có điểm TF-IDF cao nhất
    highest_tfidf_sentence = unique_sentences[highest_score_index]

    # Tìm vị trí bắt đầu (answer_start) và kết thúc (answer_end) của câu trong context ban đầu
    answer_start = context.find(highest_tfidf_sentence)
    if answer_start != -1:
        answer_end = answer_start + len(highest_tfidf_sentence)
    else:
        # Nếu không tìm thấy câu trong context ban đầu (đề phòng các trường hợp không khớp hoàn toàn)
        answer_start = -1
        answer_end = -1

    return highest_tfidf_sentence, answer_start, answer_end



In [20]:
# Áp dụng hàm lên từng hàng của DataFrame và trả về 3 giá trị
df[['Answer_of_Title', 'Answer_Start', 'Answer_End']] = df.apply(
    lambda row: pd.Series(extract_highest_tfidf_sentence_with_position(row['Title'], row['Detailed Content'])),
    axis=1
)

# Hiển thị kết quả
df[['Title', 'Detailed Content', 'Answer_of_Title', 'Answer_Start', 'Answer_End']].head()


,Title,Detailed Content,Answer_of_Title,Answer_Start,Answer_End
0,Rào cản tự nhiên chống lại nhiễm trùng về Da,Da thường chặn các vi sinh vật xâm nhập trừ kh...,Các ngoại lệ bao gồm:\nVirus Papilloma người\n...,144,501
1,Rào cản tự nhiên chống lại nhiễm trùng về Niêm...,Nhiều màng nhầy ngập trong chất tiết có đặc tí...,"Ví dụ, chất nhầy cổ tử cung, dịch tuyến tiền l...",62,519
2,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Đường hô hấp có các hệ thống lọc đường thở trê...,Đường hô hấp có các hệ thống lọc đường thở trên.,0,48
3,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Các rào cản trong đường tiêu hóa bao gồm pH ac...,Các rào cản trong đường tiêu hóa bao gồm pH ac...,0,230
4,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Các rào cản về đường sinh dục tiết niệu bao gồ...,Các rào cản về đường sinh dục tiết niệu bao gồ...,0,318


In [26]:
df[['Title', 'Detailed Content', 'Answer_of_Title', 'Answer_Start', 'Answer_End']].head(20)

,Title,Detailed Content,Answer_of_Title,Answer_Start,Answer_End
0,Rào cản tự nhiên chống lại nhiễm trùng về Da,Da thường chặn các vi sinh vật xâm nhập trừ kh...,Các ngoại lệ bao gồm:\nVirus Papilloma người\n...,144,501
1,Rào cản tự nhiên chống lại nhiễm trùng về Niêm...,Nhiều màng nhầy ngập trong chất tiết có đặc tí...,"Ví dụ, chất nhầy cổ tử cung, dịch tuyến tiền l...",62,519
2,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Đường hô hấp có các hệ thống lọc đường thở trê...,Đường hô hấp có các hệ thống lọc đường thở trên.,0,48
3,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Các rào cản trong đường tiêu hóa bao gồm pH ac...,Các rào cản trong đường tiêu hóa bao gồm pH ac...,0,230
4,Rào cản tự nhiên chống lại nhiễm trùng về Đườn...,Các rào cản về đường sinh dục tiết niệu bao gồ...,Các rào cản về đường sinh dục tiết niệu bao gồ...,0,318
5,Cơ chế phòng vệ của vật chủ chống lại sự nhiễm...,Da thường chặn các vi sinh vật xâm nhập trừ kh...,Da thường chặn các vi sinh vật xâm nhập trừ kh...,0,143
6,Cơ chế phòng vệ của vật chủ chống lại sự nhiễm...,Nhiều màng nhầy ngập trong chất tiết có đặc tí...,"Ví dụ, chất nhầy cổ tử cung, dịch tuyến tiền l...",62,519
7,Cơ chế phòng vệ của vật chủ chống lại sự nhiễm...,Đường hô hấp có các hệ thống lọc đường thở trê...,Đường hô hấp có các hệ thống lọc đường thở trên.,0,48
8,Cơ chế phòng vệ của vật chủ chống lại sự nhiễm...,Các rào cản trong đường tiêu hóa bao gồm pH ac...,Nếu nhu động ruột bị giảm (ví dụ như do thuốc ...,231,672
9,Cơ chế phòng vệ của vật chủ chống lại sự nhiễm...,Các rào cản về đường sinh dục tiết niệu bao gồ...,Các rào cản về đường sinh dục tiết niệu bao gồ...,0,318


In [27]:
# Lưu DataFrame vào file CSV
df.to_csv('/content/drive/MyDrive/ChatBox/data/medical_tf-idf.csv', index=False, encoding='utf-8')